In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from sklearn.model_selection import train_test_split , cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.compose import ColumnTransformer

In [3]:
df = pd.read_csv("../datasets/train.csv",usecols=['Age','Fare','Survived'])
df.head()

,Survived,Age,Fare
0,0,22.0,7.2500
1,1,38.0,71.2833
2,1,26.0,7.9250
3,1,35.0,53.1000
4,0,35.0,8.0500


In [4]:
df.dropna(inplace=True)

In [5]:
df.shape

(714, 3)

In [6]:
df.head()

,Survived,Age,Fare
0,0,22.0,7.2500
1,1,38.0,71.2833
2,1,26.0,7.9250
3,1,35.0,53.1000
4,0,35.0,8.0500


In [7]:
X = df.iloc[:,1:]
y = df.iloc[:,0]

In [8]:
X_train,X_test,Y_train,Y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [9]:
clf = DecisionTreeClassifier()

In [10]:
clf.fit(X_train,Y_train)
y_pred = clf.predict(X_test)

In [11]:
accuracy_score(Y_test,y_pred)

0.6363636363636364

In [12]:
np.mean(cross_val_score(DecisionTreeClassifier(),X,y,cv=10,scoring='accuracy'))

0.6260954616588419

#### Now lets use Discretixation

In [14]:
kbin_age = KBinsDiscretizer(n_bins=5,encode='ordinal',strategy='quantile')
kbin_fare = KBinsDiscretizer(n_bins=5,encode='ordinal',strategy='quantile')

In [15]:
trf = ColumnTransformer([
    ('first',kbin_age,[0]),
    ('second',kbin_fare,[1])
])

In [17]:
X_train_trf = trf.fit_transform(X_train)
X_test_trf = trf.transform(X_test)

In [18]:
trf.named_transformers_['first'].n_bins_

array([5])

In [20]:
trf.named_transformers_['first'].bin_edges_

array([array([ 0.42, 19.  , 25.  , 32.  , 42.  , 80.  ])], dtype=object)

In [21]:
output = pd.DataFrame({
    'afe': X_train['Age'],
    'age_trf':X_train_trf[:,0],
    'fare':X_train['Fare'],
    'fare_trf': X_train_trf[:,1]
})

In [22]:
output['age_labels'] = pd.cut(x=X_train['Age'],bins=trf.named_transformers_['first'].bin_edges_[0].tolist())
output['Fare_labels'] = pd.cut(x=X_train['Fare'],bins=trf.named_transformers_['second'].bin_edges_[0].tolist())

In [23]:
output.sample(10)

,afe,age_trf,fare,fare_trf,age_labels,Fare_labels
319,40.0,3.0,134.5000,4.0,"(32.0, 42.0]","(51.479, 512.329]"
789,46.0,4.0,79.2000,4.0,"(42.0, 80.0]","(51.479, 512.329]"
743,24.0,1.0,16.1000,2.0,"(19.0, 25.0]","(13.0, 26.0]"
479,2.0,0.0,12.2875,1.0,"(0.42, 19.0]","(7.896, 13.0]"
660,50.0,4.0,133.6500,4.0,"(42.0, 80.0]","(51.479, 512.329]"
722,34.0,3.0,13.0000,2.0,"(32.0, 42.0]","(7.896, 13.0]"
626,57.0,4.0,12.3500,1.0,"(42.0, 80.0]","(7.896, 13.0]"
144,18.0,0.0,11.5000,1.0,"(0.42, 19.0]","(7.896, 13.0]"
679,36.0,3.0,512.3292,4.0,"(32.0, 42.0]","(51.479, 512.329]"
453,49.0,4.0,89.1042,4.0,"(42.0, 80.0]","(51.479, 512.329]"


### Again Applying Decision Trees but on transformed Data

In [24]:
clf = DecisionTreeClassifier()
clf.fit(X_train_trf,Y_train)
y_pred_2 = clf.predict(X_test_trf)

In [25]:
accuracy_score(Y_test,y_pred_2)

0.6433566433566433

In [26]:
X_trf = trf.fit_transform(X)
np.mean(cross_val_score(DecisionTreeClassifier(),X_trf,y,cv=10,scoring='accuracy'))

0.673943661971831

#### Not that much better but remember this dataset is not quite the best example to test on